# Are-MipiA1A2 : Venin

In [78]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


#class agent :
#    def __init__(self):
#        self.efficacite= 0.1
#        self.spawn = 0.1
#        self.coagulation = 0.1
#        self.perte = 0.1
#
#globule_rouge = agent()
#globule_blanc = agent()
#plaquette = agent()
#venin = agent()
#coagulation = agent()
#anti_coagulant = agent()



#"""Type : Cell = List[float**6]"""





anti_coagulantefficacite = 0.1
globule_blancefficacite = 0.1
veninefficacite = 0.5
plaquettescoagulation = 2
veninechange=0.5
globule_blancechange=0.5

In [79]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,1)
    return matrix
            

In [80]:
def init_matrice_venin(N,x,y,Ve):
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            matrix[i,j]=(a , (1 - a)*0.023 , (1 - a)*0.977,0,(1 - a)*0.977,0,1)
    matrix[x,y,3]=Ve
    return matrix

In [81]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(n):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [82]:
def selfinteract(mat,N):
    matrix = mat.copy()
    a = [0 for i in range(7)]
    for i in range(N):
        for j in range(N):
            
            a = mat[i,j].copy()
            
            #globule_rouge+venin
            if a[3] > 0:
                a[0] = a[0] - a[3]*veninefficacite
                if a[0]<0:
                    a[0] = 0
            #globule_blanc+venin
            if a[3] > 0:
                a[3] = a[3] - a[1]*globule_blancefficacite
                a[1] = 0
                a[6] = 10
            #plaquettes+plaquettes
            a[4] = a[4] + a[2]*plaquettescoagulation
            a[2] = a[2] - a[2]/plaquettescoagulation
            #coagulation+anti_coagulant
            if a[5]>0:
                a[4] = a[4] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            #globule_blanc+coagulation
            a[4] = a[4] - a[1]*globule_blancefficacite
            if a[4] <0:
                a[4] = 0
            #venin+anti_coagulant
            if a[3] > 0 and a[5]>0:
                a[3] = a[3] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            
            #coagulation
            if a[3]>0:
                a[4]=a[4]+a[3] *veninefficacite
                if a[4]>1:
                    a[4]=1
            
            matrix[i,j] = a.copy()
    return matrix
            

In [83]:
def interact(mat,i,j,k,l):
    matr = mat.copy()
    matr[i,j,3] = mat[i,j,3] + mat[k,l,3]*veninechange - mat[i,j,3]*veninechange
    matr[k,l,3] = mat[k,l,3] + mat[j,k,3]*veninechange - mat[k,l,3]*veninechange
    
    matr[i,j,1] = mat[i,j,1] + mat[k,l,1]*globule_blancechange - mat[i,j,1]*globule_blancechange
    matr[k,l,1] = mat[k,l,1] + mat[i,j,1]*globule_blancechange - mat[k,l,1]*globule_blancechange
    return matr

In [84]:
def inter_1(N,mat):
    
    
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            if a[3] > 0 :
                k = np.random.choice(4)
                if k == 0 and i > 0:
                    return interact(mat,i,j,i-1,j)
                if k == 1 and j < N:
                    return interact(mat,i,j,i,j+1)
                if k == 2 and i < N :
                    return interact(mat,i,j,i+1,j)
                if k == 3 and j > 0:
                    return interact(mat,i,j,i,j-1)
            
            return mat

In [85]:
def inter_2(N,mat):
    for i in range(N):
        for j in range(N):
            a =  mat[i,j].copy()
            if a[3] > 0 :
                if i>0:
                    mat = interact(mat,i,j,i-1,j)
                if j<N-1:
                    mat = interact(mat,i,j,i,j+1)
                if i<N-1:
                    mat = interact(mat,i,j,i+1,j)
                if j>0:
                    mat = interact(mat,i,j,i,j-1)
    return mat
            

In [86]:
def lis_co(lis,N):
    """list[mat(cell)]*int => list(mat[float])
    convertie une liste de matrice de cellule en liste de matrice de float (Coagulation)"""
    long=len(lis)
    Lr = np.zeros((long,N,N))
    for n in range(long):
        mat = lis[n]
        for i in range(N):
            for j in range(N):
                Lr[n,i,j] = mat[i,j,4]
                
    return Lr
                

In [87]:
def main(N,x,y,Ve,n):
    mat = init_matrice_venin(N,x,y,Ve)
    rlist=np.zeros((n+1,N,N,7))
    rlist[0,:,:,:] = mat.copy()
    for i in range(n):
        mat = inter_2(N,mat)
        mat = selfinteract(mat,N)
        rlist[i+1,:,:,:] = mat[:,:,:]
    return rlist[:,:,:,4]
    

In [88]:
main(3,1,1,2,30)

array([[[0.03603431, 0.08095425, 0.11471514],
        [0.09707603, 0.05286167, 0.02088452],
        [0.03804234, 0.01716945, 0.05168588]],

       [[0.1080181 , 0.24270523, 0.34395872],
        [0.35365298, 0.25227899, 0.1075041 ],
        [0.14532136, 0.08855422, 0.19210353]],

       [[0.14401   , 0.34702669, 0.4664146 ],
        [0.48303199, 0.34352642, 0.16182073],
        [0.2164132 , 0.13952236, 0.27758811]],

       [[0.17817334, 0.41522244, 0.54711037],
        [0.5614908 , 0.40204913, 0.20330792],
        [0.26619417, 0.17952507, 0.33484903]],

       [[0.21310852, 0.46475449, 0.60391052],
        [0.61560375, 0.44591391, 0.23891956],
        [0.30598313, 0.21435321, 0.37830628]],

       [[0.24633981, 0.50468306, 0.64771936],
        [0.65772769, 0.48278435, 0.27171719],
        [0.34088633, 0.24673585, 0.41500347]],

       [[0.27822074, 0.53973367, 0.68477242],
        [0.69384599, 0.51623147, 0.3031416 ],
        [0.37336985, 0.2779447 , 0.4483696 ]],

       [[0.30927728,